In [180]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Model Evaluation Metrics
from sklearn.metrics import classification_report, accuracy_score

#Model Selection
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model  import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

#Normalization
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

In [181]:
df_train = pd.read_csv('./Data/Titanic/train.csv')
df_test = pd.read_csv('./Data/Titanic/test.csv')
df_gender = pd.read_csv('./Data/Titanic/gender_submission.csv')

In [182]:
df_train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [183]:
df_train.describe().T

,count,mean,std,min,25%,50%,75%,max
PassengerId,891.0,446.000000,257.353842,1.00,223.5000,446.0000,668.5,891.0000
Survived,891.0,0.383838,0.486592,0.00,0.0000,0.0000,1.0,1.0000
Pclass,891.0,2.308642,0.836071,1.00,2.0000,3.0000,3.0,3.0000
Age,714.0,29.699118,14.526497,0.42,20.1250,28.0000,38.0,80.0000
SibSp,891.0,0.523008,1.102743,0.00,0.0000,0.0000,1.0,8.0000
Parch,891.0,0.381594,0.806057,0.00,0.0000,0.0000,0.0,6.0000
Fare,891.0,32.204208,49.693429,0.00,7.9104,14.4542,31.0,512.3292


# Eksik Veri Doldurma İşlemi

## Train Setinde Eksik Veri Doldurma İşlemi

In [184]:
df_train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

Age ve Embarked özelliklerindeki eksik değerleri dolduracağız fakat cabin değişkenini çok fazla eksik değerinin
olması ve dolu değerlerinin de bir kısmının anlamsız veriler içermesi sebebiyle atacağız

In [185]:
df_train['Age'].mean()

29.69911764705882

In [186]:
#Yaş değişkeninin eksik değerlerini ortalama yaş değeri ile dolduruyoruz
df_train['Age'].fillna(df_train['Age'].mean(),inplace=True)

In [187]:
df_train.Age.isnull().sum()

0

In [188]:
df_train['Embarked'].mode()

0    S
Name: Embarked, dtype: object

In [189]:
df_train['Embarked'].fillna(df_train['Embarked'].mode()[0],inplace=True)

In [190]:
df_train.Embarked.isnull().sum()

0

## Test Setinde Eksik Veri Doldurma İşlemi

In [191]:
df_test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

Age ve Fare Özelliklerindeki eksik değerleri dolduracağız fakat cabin değişkenini çok fazla eksik değerinin
olması ve dolu değerlerinin de bir kısmının anlamsız veriler içermesi sebebiyle atacağız

In [192]:
df_test['Age'].mean()

30.272590361445783

In [193]:
#Yaş değişkeninin eksik değerlerini ortalama yaş değeri ile dolduruyoruz
df_test['Age'].fillna(df_test['Age'].mean(),inplace=True)

In [194]:
df_test.Age.isnull().sum()

0

In [195]:
df_test['Fare'].mean()

35.627188489208635

In [196]:
df_test['Fare'].fillna(df_test['Fare'].mean(),inplace=True)

In [197]:
df_test.Fare.isnull().sum()

0

# Aykırı Gözlemlerin Temizlenmesi

## Train Setinde Aykırı Gözlemlerin Temizlenmesi

In [198]:
df_train.describe().T

,count,mean,std,min,25%,50%,75%,max
PassengerId,891.0,446.000000,257.353842,1.00,223.5000,446.000000,668.5,891.0000
Survived,891.0,0.383838,0.486592,0.00,0.0000,0.000000,1.0,1.0000
Pclass,891.0,2.308642,0.836071,1.00,2.0000,3.000000,3.0,3.0000
Age,891.0,29.699118,13.002015,0.42,22.0000,29.699118,35.0,80.0000
SibSp,891.0,0.523008,1.102743,0.00,0.0000,0.000000,1.0,8.0000
Parch,891.0,0.381594,0.806057,0.00,0.0000,0.000000,0.0,6.0000
Fare,891.0,32.204208,49.693429,0.00,7.9104,14.454200,31.0,512.3292


### Fare İçin Aykırı Gözlem Kontrolü ve Temizliği

In [199]:
#Fare için IQR değeri hesaplanıyor
IQR = df_train.Fare.quantile(0.75) - df_train.Fare.quantile(0.25)
IQR

23.0896

In [200]:
#### Olabildiğince en uçlardaki aykırı değerleri yakalayabilmek için IQR*3 kadar ekliyor veya çıkarıyoruz
#sinir yazan 
alt_sınır=df_train['Fare'].quantile(0.25)-(IQR*3)
üst_sınır=df_train['Fare'].quantile(0.75)+(IQR*3)
alt_sınır, üst_sınır

(-61.358399999999996, 100.2688)

In [201]:
# Fare için Alt sınırın altında gözlem yok
df_train[df_train['Fare']<alt_sınır]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked


In [202]:
#Fare değişkeni Üst Sınırın üstünde 53 gözlem bulunuyor
df_train[df_train['Fare']>üst_sınır].shape

(53, 12)

In [203]:
# Üst sınırın üstünde olan fare değişkenlerini üst sınıra baskılıyoruz
df_train.loc[df_train['Fare']>üst_sınır,'Fare'] = üst_sınır

In [204]:
# Fare değişkeni için aykırı gözlemler baskılanmış oldu
df_train[df_train['Fare']>üst_sınır].shape

(0, 12)

### Age İçin Aykırı Gözlem Kontrolü ve Temizliği

In [205]:
#Age için IQR değeri hesaplanıyor
IQR = df_train.Age.quantile(0.75) - df_train.Age.quantile(0.25)
IQR

13.0

In [206]:
#### Olabildiğince en uçlardaki aykırı değerleri yakalayabilmek için IQR*3 kadar ekliyor veya çıkarıyoruz
#sinir yazan 
alt_sınır=df_train['Age'].quantile(0.25)-(IQR*3)
üst_sınır=df_train['Age'].quantile(0.75)+(IQR*3)
alt_sınır, üst_sınır

(-17.0, 74.0)

In [207]:
# Age için Alt sınırın altında gözlem yok
df_train[df_train['Age']<alt_sınır]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked


In [208]:
#Age değişkeni Üst Sınırın üstünde 53 gözlem bulunuyor
df_train[df_train['Age']>üst_sınır].shape

(1, 12)

In [209]:
# Üst sınırın üstünde olan Age değişkenlerini üst sınıra baskılıyoruz
df_train.loc[df_train['Age']>üst_sınır,'Age'] = üst_sınır

In [210]:
# Age değişkeni için aykırı gözlemler baskılanmış oldu
df_train[df_train['Age']>üst_sınır].shape

(0, 12)

## Test Setinde Aykırı Gözlemlerin Temizlenmesi

### Fare İçin Aykırı Gözlem Kontrolü ve Temizliği

In [211]:
#Fare için IQR değeri hesaplanıyor
IQR = df_test.Fare.quantile(0.75) - df_test.Fare.quantile(0.25)
IQR

23.6042

In [212]:
#### Olabildiğince en uçlardaki aykırı değerleri yakalayabilmek için IQR*3 kadar ekliyor veya çıkarıyoruz
#sinir yazan 
alt_sınır=df_test['Fare'].quantile(0.25)-(IQR*3)
üst_sınır=df_test['Fare'].quantile(0.75)+(IQR*3)
alt_sınır, üst_sınır

(-62.9168, 102.3126)

In [213]:
# Fare için Alt sınırın altında gözlem yok
df_test[df_test['Fare']<alt_sınır]

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked


In [214]:
#Fare değişkeni Üst Sınırın üstünde 53 gözlem bulunuyor
df_test[df_test['Fare']>üst_sınır].shape

(31, 11)

In [215]:
# Üst sınırın üstünde olan fare değişkenlerini üst sınıra baskılıyoruz
df_test.loc[df_test['Fare']>üst_sınır,'Fare'] = üst_sınır

In [216]:
# Fare değişkeni için aykırı gözlemler baskılanmış oldu
df_test[df_test['Fare']>üst_sınır].shape

(0, 11)

### Age İçin Aykırı Gözlem Kontrolü ve Temizliği

In [217]:
#Age için IQR değeri hesaplanıyor
IQR = df_test.Age.quantile(0.75) - df_test.Age.quantile(0.25)
IQR

12.75

In [218]:
#### Olabildiğince en uçlardaki aykırı değerleri yakalayabilmek için IQR*3 kadar ekliyor veya çıkarıyoruz
#sinir yazan 
alt_sınır=df_test['Age'].quantile(0.25)-(IQR*3)
üst_sınır=df_test['Age'].quantile(0.75)+(IQR*3)
alt_sınır, üst_sınır

(-15.25, 74.0)

In [219]:
# Age için Alt sınırın altında gözlem yok
df_test[df_test['Age']<alt_sınır]

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked


In [220]:
#Age değişkeni Üst Sınırın üstünde 53 gözlem bulunuyor
df_test[df_test['Age']>üst_sınır].shape

(1, 11)

In [221]:
# Üst sınırın üstünde olan Age değişkenlerini üst sınıra baskılıyoruz
df_test.loc[df_test['Age']>üst_sınır,'Age'] = üst_sınır

In [222]:
# Age değişkeni için aykırı gözlemler baskılanmış oldu
df_test[df_test['Age']>üst_sınır].shape

(0, 11)

# Gereksiz Özelliklerin Atılması

In [223]:
df_train.drop(columns=['PassengerId','Cabin','Ticket','Name'],inplace=True)
df_test.drop(columns=['PassengerId','Cabin','Ticket','Name'],inplace=True)

In [224]:
df_train

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.000000,1,0,7.2500,S
1,1,1,female,38.000000,1,0,71.2833,C
2,1,3,female,26.000000,0,0,7.9250,S
3,1,1,female,35.000000,1,0,53.1000,S
4,0,3,male,35.000000,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,0,2,male,27.000000,0,0,13.0000,S
887,1,1,female,19.000000,0,0,30.0000,S
888,0,3,female,29.699118,1,2,23.4500,S
889,1,1,male,26.000000,0,0,30.0000,C


In [225]:
df_test

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,34.50000,0,0,7.8292,Q
1,3,female,47.00000,1,0,7.0000,S
2,2,male,62.00000,0,0,9.6875,Q
3,3,male,27.00000,0,0,8.6625,S
4,3,female,22.00000,1,1,12.2875,S
...,...,...,...,...,...,...,...
413,3,male,30.27259,0,0,8.0500,S
414,1,female,39.00000,0,0,102.3126,C
415,3,male,38.50000,0,0,7.2500,S
416,3,male,30.27259,0,0,8.0500,S


In [226]:
# train_test_split fonksiyonunun ayırdığı gibi verimizi bağımlı değişken ve bağımsız değişkenlerine ayırıyoruz.
X_train = df_train.drop(columns=['Survived'])
y_train = df_train['Survived']
X_test = df_test
y_test = df_gender['Survived']

Buradan sonra
- Kategorik değişkenler sayısal değişkenlere dönüştürülecek
- GridSearchCV kullanılarak LogisticRegression ve KNeighborsClassifier'ın classification_report sonuçlarına bakılacak.

In [227]:
df_encoded_train = pd.get_dummies(X_train,columns=['Sex','Embarked'], drop_first=True, dtype=np.int64)
df_encoded_test = pd.get_dummies(df_test,columns=['Sex','Embarked'], drop_first=True, dtype=np.int64)

In [228]:
df_encoded_train

,Pclass,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S
0,3,22.000000,1,0,7.2500,1,0,1
1,1,38.000000,1,0,71.2833,0,0,0
2,3,26.000000,0,0,7.9250,0,0,1
3,1,35.000000,1,0,53.1000,0,0,1
4,3,35.000000,0,0,8.0500,1,0,1
...,...,...,...,...,...,...,...,...
886,2,27.000000,0,0,13.0000,1,0,1
887,1,19.000000,0,0,30.0000,0,0,1
888,3,29.699118,1,2,23.4500,0,0,1
889,1,26.000000,0,0,30.0000,1,0,0


# Normalizasyon ve Standardizasyon

In [229]:
#Normalizasyon 0 ile 1 arasına dönüştürür
# Aykırı değerlere duyarlıdır. Eğer kullanılacaksa aykırı değerlerle başa çıkmanın yolu bulunmalıdır
# Özelliklerin sıralamasının veya orjinal dağılımının önemli olduğu durumlarda MinMaxScaler kullanılabilir.
# KNN , SVM
mscaler = MinMaxScaler()
df_encoded_train = mscaler.fit_transform(df_encoded_train)
df_encoded_test = mscaler.transform(df_encoded_test)

In [236]:
df_encoded_train

array([[1.        , 0.29328622, 0.125     , ..., 1.        , 0.        ,
        1.        ],
       [0.        , 0.51073661, 0.125     , ..., 0.        , 0.        ,
        0.        ],
       [1.        , 0.34764882, 0.        , ..., 0.        , 0.        ,
        1.        ],
       ...,
       [1.        , 0.39792223, 0.125     , ..., 0.        , 0.        ,
        1.        ],
       [0.        , 0.34764882, 0.        , ..., 1.        , 0.        ,
        0.        ],
       [1.        , 0.42919272, 0.        , ..., 1.        , 1.        ,
        0.        ]])

In [178]:
#Standardizasyon -1 ile 1
# Aykırı değerlere karşı Min_maxScaler'dan daha dayanıklıdır.
# Verilerin dağılımını normal dağılıma dönüştürür. Bu verilerin normal dağılımla dağıldığını kabul 
# eden modeller için önemlidir.
# Lineer Modeller, Gaussian Naive Bayes
scaler = StandardScaler()
df_encoded_train = scaler.fit_transform(df_encoded_train)
df_encoded_test = scaler.transform(df_encoded_test)

# Naive Bayes ile Titanic Tahminleri

In [230]:
from sklearn.naive_bayes import GaussianNB
nb_clf = GaussianNB()
NB_clf_params = {'var_smoothing': np.logspace(0,-9, num=100)}
nb_clf_cv = GridSearchCV(nb_clf,param_grid=NB_clf_params,cv=5,verbose=3)
nb_clf_cv.fit(df_encoded_train,y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV 1/5] END .................var_smoothing=1.0;, score=0.726 total time=   0.0s
[CV 2/5] END .................var_smoothing=1.0;, score=0.815 total time=   0.0s
[CV 3/5] END .................var_smoothing=1.0;, score=0.815 total time=   0.0s
[CV 4/5] END .................var_smoothing=1.0;, score=0.781 total time=   0.0s
[CV 5/5] END .................var_smoothing=1.0;, score=0.826 total time=   0.0s
[CV 1/5] END ..var_smoothing=0.8111308307896871;, score=0.726 total time=   0.0s
[CV 2/5] END ..var_smoothing=0.8111308307896871;, score=0.815 total time=   0.0s
[CV 3/5] END ..var_smoothing=0.8111308307896871;, score=0.826 total time=   0.0s
[CV 4/5] END ..var_smoothing=0.8111308307896871;, score=0.764 total time=   0.0s
[CV 5/5] END ..var_smoothing=0.8111308307896871;, score=0.820 total time=   0.0s
[CV 1/5] END ...var_smoothing=0.657933224657568;, score=0.726 total time=   0.0s
[CV 2/5] END ...var_smoothing=0.65793322465756

GridSearchCV(cv=5, estimator=GaussianNB(),
             param_grid={'var_smoothing': array([1.00000000e+00, 8.11130831e-01, 6.57933225e-01, 5.33669923e-01,
       4.32876128e-01, 3.51119173e-01, 2.84803587e-01, 2.31012970e-01,
       1.87381742e-01, 1.51991108e-01, 1.23284674e-01, 1.00000000e-01,
       8.11130831e-02, 6.57933225e-02, 5.33669923e-02, 4.32876128e-02,
       3.51119173e-02, 2.84803587e-02, 2.31...
       1.23284674e-07, 1.00000000e-07, 8.11130831e-08, 6.57933225e-08,
       5.33669923e-08, 4.32876128e-08, 3.51119173e-08, 2.84803587e-08,
       2.31012970e-08, 1.87381742e-08, 1.51991108e-08, 1.23284674e-08,
       1.00000000e-08, 8.11130831e-09, 6.57933225e-09, 5.33669923e-09,
       4.32876128e-09, 3.51119173e-09, 2.84803587e-09, 2.31012970e-09,
       1.87381742e-09, 1.51991108e-09, 1.23284674e-09, 1.00000000e-09])},
             verbose=3)

In [231]:
nb_clf_cv.best_params_

{'var_smoothing': 1.0}

In [232]:
y_pred = nb_clf_cv.predict(df_encoded_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.87      0.97      0.91       266
           1       0.93      0.74      0.82       152

    accuracy                           0.88       418
   macro avg       0.90      0.85      0.87       418
weighted avg       0.89      0.88      0.88       418



In [233]:
accuracy_score(y_test,y_pred)

0.8827751196172249

In [234]:
y_pred_tr = nb_clf_cv.predict(df_encoded_train)
print(classification_report(y_train,y_pred_tr))

              precision    recall  f1-score   support

           0       0.79      0.94      0.86       549
           1       0.85      0.60      0.70       342

    accuracy                           0.81       891
   macro avg       0.82      0.77      0.78       891
weighted avg       0.81      0.81      0.80       891



# Topluluk Modeli ile Titanic Tahminleri

In [50]:
from sklearn.ensemble import VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier

In [51]:
log_reg = LogisticRegression()
log_reg_params = {'penalty':['l1','l2','elasticnet'],
          'C':[1,2,3,4,5,6,10,20,30,40,50],
          'max_iter':[100,200,300]}

In [52]:
knn_clf = KNeighborsClassifier()
knn_clf_params = {
    'n_neighbors': [i for i in range(1,15)],
    'metric': ['euclidean','minkowski','manhattan','cosine']
}

In [53]:
dtree_clf = DecisionTreeClassifier()
dtree_clf_params = {
    'max_depth': [3,4,5,7,9],
    'min_samples_split':[2,5,8,10,20],
    'min_samples_leaf': [1,2,3,4],
    'criterion' : ['gini','entropy']
}

In [54]:
nb_clf = GaussianNB()
nb_clf_params = {'var_smoothing': np.logspace(0,-9, num=100)}

In [55]:
knn_clf_grid = GridSearchCV(knn_clf,knn_clf_params,cv=5,n_jobs=-1,verbose=1)
logreg_clf_grid = GridSearchCV(log_reg,log_reg_params,cv=5,n_jobs=-1,verbose=2)
dtree_clf_grid = GridSearchCV(dtree_clf,dtree_clf_params,cv=5,n_jobs=-1,verbose=3)
nb_clf_grid = GridSearchCV(nb_clf,nb_clf_params,cv=5,n_jobs=-1,verbose=3)

In [56]:
voting_clf = VotingClassifier(
               [('knn',knn_clf_grid),
                ('log_reg',logreg_clf_grid),
                ('dtree',dtree_clf_grid),
                ('nb',nb_clf_grid)],
    voting='hard'
)

In [58]:
voting_clf.fit(df_encoded_train,y_train)

Fitting 5 folds for each of 56 candidates, totalling 280 fits
Fitting 5 folds for each of 99 candidates, totalling 495 fits
Fitting 5 folds for each of 200 candidates, totalling 1000 fits
Fitting 5 folds for each of 100 candidates, totalling 500 fits


VotingClassifier(estimators=[('knn',
                              GridSearchCV(cv=5,
                                           estimator=KNeighborsClassifier(),
                                           n_jobs=-1,
                                           param_grid={'metric': ['euclidean',
                                                                  'minkowski',
                                                                  'manhattan',
                                                                  'cosine'],
                                                       'n_neighbors': [1, 2, 3,
                                                                       4, 5, 6,
                                                                       7, 8, 9,
                                                                       10, 11,
                                                                       12, 13,
                                                                       14]},
                                           verbose=1)),
                             ('log_reg',
                              GridSearchCV(cv=5, estimator=LogisticRegression(),
                                           n_jobs=-1,
                                           param_grid={'C': [1, 2, 3, 4, 5, 6,
                                                             10, 20, 30, 40,
                                                             50],
                                                       'max_ite...
       1.23284674e-07, 1.00000000e-07, 8.11130831e-08, 6.57933225e-08,
       5.33669923e-08, 4.32876128e-08, 3.51119173e-08, 2.84803587e-08,
       2.31012970e-08, 1.87381742e-08, 1.51991108e-08, 1.23284674e-08,
       1.00000000e-08, 8.11130831e-09, 6.57933225e-09, 5.33669923e-09,
       4.32876128e-09, 3.51119173e-09, 2.84803587e-09, 2.31012970e-09,
       1.87381742e-09, 1.51991108e-09, 1.23284674e-09, 1.00000000e-09])},
                                           verbose=3))])

In [59]:
y_pred = voting_clf.predict(df_encoded_test)
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       0.97      0.87      0.92       297
           1       0.74      0.93      0.83       121

    accuracy                           0.89       418
   macro avg       0.86      0.90      0.87       418
weighted avg       0.90      0.89      0.89       418



In [60]:
y_pred_tr = voting_clf.predict(df_encoded_train)
print(classification_report(y_pred_tr,y_train))

              precision    recall  f1-score   support

           0       0.96      0.81      0.88       655
           1       0.63      0.92      0.75       236

    accuracy                           0.84       891
   macro avg       0.80      0.86      0.81       891
weighted avg       0.88      0.84      0.84       891



In [61]:
voting_clf2 = VotingClassifier(
               [('knn',knn_clf_grid),
                ('log_reg',logreg_clf_grid),
                ('dtree',dtree_clf_grid),
                ('nb',nb_clf_grid)],
    voting='soft'
)

In [62]:
voting_clf2.fit(df_encoded_train,y_train)

Fitting 5 folds for each of 56 candidates, totalling 280 fits
Fitting 5 folds for each of 99 candidates, totalling 495 fits
Fitting 5 folds for each of 200 candidates, totalling 1000 fits
Fitting 5 folds for each of 100 candidates, totalling 500 fits


VotingClassifier(estimators=[('knn',
                              GridSearchCV(cv=5,
                                           estimator=KNeighborsClassifier(),
                                           n_jobs=-1,
                                           param_grid={'metric': ['euclidean',
                                                                  'minkowski',
                                                                  'manhattan',
                                                                  'cosine'],
                                                       'n_neighbors': [1, 2, 3,
                                                                       4, 5, 6,
                                                                       7, 8, 9,
                                                                       10, 11,
                                                                       12, 13,
                                                                       14]},
                                           verbose=1)),
                             ('log_reg',
                              GridSearchCV(cv=5, estimator=LogisticRegression(),
                                           n_jobs=-1,
                                           param_grid={'C': [1, 2, 3, 4, 5, 6,
                                                             10, 20, 30, 40,
                                                             50],
                                                       'max_ite...
       1.23284674e-07, 1.00000000e-07, 8.11130831e-08, 6.57933225e-08,
       5.33669923e-08, 4.32876128e-08, 3.51119173e-08, 2.84803587e-08,
       2.31012970e-08, 1.87381742e-08, 1.51991108e-08, 1.23284674e-08,
       1.00000000e-08, 8.11130831e-09, 6.57933225e-09, 5.33669923e-09,
       4.32876128e-09, 3.51119173e-09, 2.84803587e-09, 2.31012970e-09,
       1.87381742e-09, 1.51991108e-09, 1.23284674e-09, 1.00000000e-09])},
                                           verbose=3))],
                 voting='soft')

In [63]:
y_pred2 = voting_clf2.predict(df_encoded_test)
print(classification_report(y_pred2,y_test))

              precision    recall  f1-score   support

           0       0.87      0.91      0.89       256
           1       0.84      0.79      0.82       162

    accuracy                           0.86       418
   macro avg       0.86      0.85      0.85       418
weighted avg       0.86      0.86      0.86       418



In [64]:
y_pred2_tr = voting_clf.predict(df_encoded_train)
print(classification_report(y_pred2_tr,y_train))

              precision    recall  f1-score   support

           0       0.96      0.81      0.88       655
           1       0.63      0.92      0.75       236

    accuracy                           0.84       891
   macro avg       0.80      0.86      0.81       891
weighted avg       0.88      0.84      0.84       891

